# 4.1. 리소스 클린업

# 1. 환경 설정

In [57]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1.0 기존의 변수 불러오기

In [58]:
%store -r code_pipeline_serving_config_json_path
%store -r sm_pipeline_serving_config_json_path

from common_utils import load_json

code_pipeline_serving_dict = load_json(code_pipeline_serving_config_json_path)
sm_pipeline_serving_dict = load_json(sm_pipeline_serving_config_json_path)

import json
print("Code Pipeline Series Params: ")
print (json.dumps(code_pipeline_serving_dict, indent=2))
print("SageMaker Pipeline Series Params: ")
print (json.dumps(sm_pipeline_serving_dict, indent=2))



Code Pipeline Series Params: 
{
  "code_pipeline_role_arn": "arn:aws:iam::057716757052:role/code-pipeline-gsmoon",
  "code_build_service_arn": "arn:aws:iam::057716757052:role/codebuild-gsmoon",
  "project_prefix": "CodePipeline-Serving-NCF",
  "region": "us-east-1",
  "account_id": "057716757052",
  "serving_code_repo_name": "ncf-serving",
  "code_build_project_name": "ncf-serving-sm-pipeline",
  "bucket": "sagemaker-us-east-1-057716757052",
  "code_pipeline_name": "ncf-serving-code-pipeline",
  "model_package_group_name": "NCF-Model-CodePipeline",
  "endpoint_name": "ncf-codepipeline-endpoint",
  "branch_name": "master"
}
SageMaker Pipeline Series Params: 
{
  "sm_pipeline_name": "sm-serving-pipeline",
  "endpoint_instance_type": "ml.g4dn.xlarge",
  "ModelApprovalStatus": "Approved",
  "bucket": "sagemaker-us-east-1-057716757052"
}


In [59]:
import sagemaker
import boto3

repository_name = code_pipeline_serving_dict["serving_code_repo_name"]
code_build_project_name = code_pipeline_serving_dict["code_build_project_name"]
code_pipeline_name = code_pipeline_serving_dict["code_pipeline_name"]
endpoint_name = code_pipeline_serving_dict["endpoint_name"]
bucket = code_pipeline_serving_dict["bucket"]
model_package_group_name = code_pipeline_serving_dict["model_package_group_name"]

sm_pipeline_name = sm_pipeline_serving_dict["sm_pipeline_name"]

print("repository_name: ", repository_name)
print("bucket: ", bucket)
print("code_build_project_name: ", code_build_project_name)
print("sm_pipeline_name: ", sm_pipeline_name)
print("model_package_group_name: ", model_package_group_name)
print("endpoint_name: ", endpoint_name)

repository_name:  ncf-serving
bucket:  sagemaker-us-east-1-057716757052
code_build_project_name:  ncf-serving-sm-pipeline
sm_pipeline_name:  sm-serving-pipeline
model_package_group_name:  NCF-Model-CodePipeline
endpoint_name:  ncf-codepipeline-endpoint


# 2. 코드 리파지토리 삭제

In [61]:
%%sh -s {repository_name}
repository_name=$1
aws codecommit delete-repository --repository-name $repository_name 


{
    "repositoryId": "0d2debd8-014e-4f0d-9151-a05bee294dff"
}


# 3. 코드 빌드 프로젝트 삭제

In [62]:
%%sh -s {code_build_project_name}
code_build_project_name=$1
aws codebuild delete-project --name $code_build_project_name

# 4. 코드 파이프라인 삭제

In [63]:
%%sh -s {code_pipeline_name}
pipeline_name=$1
aws codepipeline delete-pipeline --name $pipeline_name

# 5. 엔드포인트, 엔드포인트 컨피그, 세이즈 메이커 모델 삭제

In [64]:
import boto3

sm_client = boto3.client('sagemaker')
from inference_utils import delete_endpoint
delete_endpoint(client = sm_client, 
                endpoint_name = endpoint_name
               )



#### Start
--- Deleted model: pipelines-s72ob9htsxmg-MyModelCreationStep--z8paSw5N5N
--- Deleted endpoint: ncf-codepipeline-endpoint
--- Deleted endpoint_config: ncf-codepipeline-endpoint-11-20-12-11-13


# 5. SageMaker Model Building Pipeline 삭제




## 5.1 파이프라인 삭제

- 파이프라인 이름 검색

sm_pipeline_name 의 변수 이름으로 시작되는 파이프라인 정보 검색

In [ ]:
import boto3

client = boto3.client('sagemaker')

response = client.list_pipelines(
    PipelineNamePrefix= sm_pipeline_name,
    SortBy='Name',
    SortOrder='Descending',
    MaxResults=1
)

response

In [ ]:
response = client.delete_pipeline(
    PipelineName=sm_pipeline_name,
)
response

## 5.2. 파이프라인 관련 S3 내 파일 삭제

In [ ]:
# ! aws s3 ls {bucket}/{sm_pipeline_name} --recursive
! aws s3 rm s3://{bucket}/{sm_pipeline_name} --recursive

# 6. 모델 레지스트리 삭제

- 자세한 것은 개발자 가이드 참고 하세요. [Boto3 SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.delete_model_package)

## 6.1. 전체 model package group 검색
- NameContains 의 이름 있는 것만 검색

In [ ]:
response = client.list_model_package_groups(
    NameContains = 'mimick',
    SortBy='CreationTime',
    SortOrder='Ascending'
)
response

## 6.2. 기술된 model package group 안애서 등록된 model package 만 검색

In [ ]:
model_package_group_name = sm_pipeline_name

In [ ]:
response = client.list_model_packages(
    ModelPackageGroupName=model_package_group_name , 
    SortBy='CreationTime',
    SortOrder='Ascending'
)
response

## 6.3. model package 삭제

In [ ]:
for e in response['ModelPackageSummaryList']:
    print(e)
    ModelPackageArn = e['ModelPackageArn']
    print(f"########## Delete: {ModelPackageArn}")
    response = client.delete_model_package(
        ModelPackageName=ModelPackageArn
    )    
    print(response)

## 6.4. 기술된 model package group 삭제

In [ ]:
response = client.delete_model_package_group(
    ModelPackageGroupName= model_package_group_name
)
response